In [1]:
:opt no-lint
{-# LANGUAGE ScopedTypeVariables TypeApplications RankNTypes #-}

## 계산이 끝나지 않는 람다식
람다식 작은걸음 $(\lambda x.x\;x)\;e_2 \longmapsto e_2\;e_2$를 생각해 보자.
여기서 $e_2$가 적용하는 함수 $(\lambda x.x\;x)$와 똑같은 식인 경우에 어떤 일이 벌어질까?
바로 아래와 같이 된다.
\newline\indent
$(\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto (\lambda x.x\;x)(\lambda x.x\;x) \longmapsto \ldots$
\newline
$\beta$규칙에 따라 함수 몸체 $x\;x$에서 $x$를 $(\lambda x.x\;x)$로 바꿔치면
원래 처음 시작했던 식인 $(\lambda x.x\;x)(\lambda x.x\;x)$가 되어버린다.
그래서 아무리 여러 번 $\beta$줄임을 반복해도 제자리걸음만 할 뿐 계산이 끝나지 않는다.
이제 드브루인 인덱스(그림 \ref{fig:ChurchDeBruijn})를 활용해 타입없는 람다계산법에서 끝나지
않는 계산을 하스켈로 다뤄보자. 우선 람다계산법의 문법구조를 아래와 같은
데이터 타입으로 선언한다.

In [2]:
data DExpr = DI Int | DAbs DExpr | DApp DExpr DExpr deriving (Eq,Ord)

In [3]:
-- GHC가 자동으로 deriving하는 Show 인스턴스가 아닌  더 간략한 출력을 위해
instance Show DExpr where   -- 직접 작성한 DExpr에 대한 Show 인스턴스
  showsPrec _ (DI i) = showString (show i)
  showsPrec p (DAbs e) = showParen (p > 1) $
              showString "\\" . showsPrec 1 e
  showsPrec p (DApp e1 e2) = showParen (p > 9) $
              showsPrec 9 e1 . showString " " . showsPrec 10 e2

In [4]:
DAbs (DAbs (DI 1 `DApp` DI 2))        -- \x.\y.x y
DAbs (DI 1) `DApp` DAbs (DI 1)        -- (\x.x)(\x.x)
DAbs ((DI 1 `DApp` DI 1) `DApp` DI 1) -- \x.x x x
DAbs (DI 1 `DApp` (DI 1 `DApp` DI 1)) -- \x.x (x x)

\\1 2

(\1) (\1)

\1 1 1

\1 (1 1)

\noindent
그리고 매개변수에 넘겨받은 인자를 자기 자신에 적용(self-appliation)하는 함수 $\lambda x.x\;x$는
드브루인 인덱스를 활용한 표현으로는 $\lambda 1~1$이다. 이를 `w`로 선언해 놓고 앞으로 의미구조를
하스켈로 구현하여 `w w`가 제자리걸음함을 확인해 보자.

In [5]:
w = DAbs (DI 1 `DApp` DI 1) -- \x.x x

w         -- self-application 함수인 w
DApp w w  -- w를 자기 자신에 적용한 식

\1 1

(\1 1) (\1 1)

작은걸음 의미구조는 $\beta$규칙 및 맥락 규칙들로 정의할 수 있다.
\newline\vspace*{-2ex}
$$(\lambda\,e)\;e_2 \longmapsto \;\downarrow_0^1 \{1{\mapsto}(\uparrow_0^1 e_2)\}e$$
$$\frac{~ e_1\longmapsto e_1'~}{~ e_1~e_2\longmapsto e_1'~e_2 ~}
\qquad\frac{~ e_2\longmapsto e_2'~}{~ e_1~e_2\longmapsto e_1~e_2' ~}
\qquad\frac{~ e\longmapsto e' ~}{~ \lambda\,e \longmapsto \lambda\,e' ~}
$$
위에서 $\downarrow_m^k$, $\uparrow_m^k$와 바꿔치기 $\{i{\mapsto}e\}$가 어떻게 동작하는지만 확실히 하면
나머지 맥락 규칙에 대한 처리는 산술식 작은걸음 계산기인 `step`을 작성하듯 선언하면 된다.
$\downarrow_m^k = \uparrow_m^{-k}$이며, 일반적으로 $\downarrow_m^k e$는 식 $e$에서
$m$보다 큰 드브루인 인덱스를 $k$만큼씩 증가시킨 식을 나타내는 표현으로 왼쪽 아래처럼 정의할 수 있다.
함수요약식에 대한 정의에서 함수 전체에는 $m$이다가 함수 몸체에서 $m+1$이
되는 이유는 바인더 $\lambda$가 한겹 더 감싸진 몸체 안에서는 그 바깥에서와
같은 대상을 나타내는 드브루인 인덱스가 1씩  더 큰 수가 되기 때문이다.
바꿔치기 $\{i{\mapsto}e\}$는 $\uparrow_m^k$ 표현을 활용해 오른쪽 아래와 같이 정의할 수 있다.
$$\begin{array}{ll}
\uparrow_m^k i 
\!\!\!\!&= \begin{cases}i+k & (i>m) \\ i & (i\le m)\end{cases} \\
\uparrow_m^k (\lambda\,e)
\!\!\!\!&=~ \lambda\,(\uparrow_{m{+}1}^k e) \\
\uparrow_m^k (e_1\;e_2)
\!\!\!\!&=~ (\uparrow_m^k e_1)\;(\uparrow_m^k e_2)
\end{array}
\qquad~~
\begin{array}{ll}
\{i{\mapsto}e\}j 
\!\!\!\!&= \begin{cases}e & (i=j) \\ j & (i\neq j)\end{cases}\\
\{i{\mapsto}e\}(\lambda\, e_1) 
\!\!\!\!&=~ \lambda\, \left\{(i{+}1){\mapsto}(\uparrow_0^1 e\right)\}e_1 \\
\{i{\mapsto}e\}(e_1\;e_2)
\!\!\!\!&=~ \{i{\mapsto}e\}e_1~\{i{\mapsto}e\}e_2 \\
\end{array}$$
위 정의를 하스켈로 아래와 옮기면 다음과 같다.

In [6]:
up m k (DI i)       = DI (if i>m then i+k else i)
up m k (DAbs e)     = DAbs (up (m+1) k e)
up m k (DApp e1 e2) = DApp (up m k e1) (up m k e2)

dn m k = up m (-k)

subst i e (DI j)       = if i==j then e else DI j
subst i e (DAbs e1)    = DAbs (subst (i+1) (up 0 1 e) e1) 
subst i e (DApp e1 e2) = DApp (subst i e e1) (subst i e e2)

In [7]:
:type up
:type dn
:type subst

up :: Int -> Int -> DExpr -> DExpr

dn :: Int -> Int -> DExpr -> DExpr

subst :: Int -> DExpr -> DExpr -> DExpr

\noindent
위의 `up`, `dn`, `subst`를 활용해 다음과 같이 하스켈로 $\beta$규칙을 작성할 수 있다.

In [8]:
beta :: DExpr -> [DExpr]  -- 일단 베타 규칙만 작성
beta (DApp (DAbs e) e2) = [ dn 0 1 (subst 1 (up 0 1 e2) e) ]
beta _                  = []

In [9]:
DApp w w        -- (\x.x) (\x.x)
beta (DApp w w) -- 제자리걸음 확인!

(\1 1) (\1 1)

[(\1 1) (\1 1)]

In [10]:
step :: DExpr -> [DExpr]
step (DI _)       = []
step (DApp e1 e2) = undefined -- undefined 대신 의미규칙대로
step (DAbs e)     = undefined -- step 함수를 완성하는 연습문제

In [11]:
step :: DExpr -> [DExpr]
step (DI _)       = []
step (DApp e1 e2) = beta (DApp e1 e2)
                 ++ [DApp e1' e2 | e1' <- step e1]
                 ++ [DApp e1 e2' | e2' <- step e2]
step (DAbs e)     = [DAbs e' | e' <- step e]

In [12]:
-- step이 제대로 동작하는지 다음 내용으로 테스트해 보라
eee1 = DAbs . DAbs $ DI 1 `DApp` DI 2 `DApp` DI 3
eee2 = DAbs . DAbs $ DI 1 `DApp` DI 3
DAbs $ DApp eee1 eee2 -- \x.(\y.\z.z y x) (\w.\v.v x)
step it               -- \x.\y.\z.z (\w.\v x) x

\(\\1 2 3) (\\1 3)

[\\1 (\\1 4) 2]

## 타입없는 람다계산법에서 고정점(fixpoint)
수학에서 함수 $f$의 고정점(fixpoint)이란 $x \equiv f(x)$인 $x$를 말한다.
타입없는 람다계산법에는 아무 함수에 적용해 그 함수의 고정점을 계산하는
고정점 조합자(fixpoint combinator)가 여럿 존재한다.
하스켈 커리가 발견한 $Y=\lambda f.(\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))$는
Y조합자(Y-combinator)라 불리는 람다계산법의 대표적인 고정점 조합자로,
다음과 같이 $Y\,f \equiv_\beta f(Y\,f)$를 만족함 보일 수 있다.\vspace*{-2ex}
\begin{align*}
Y\,f \longmapsto (\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x)) \longmapsto f\,((\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))) &\\
f(Y\,f) \longmapsto f\,((\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x))) &
\end{align*}
이런 고정점의 성질을 활용하면
$Y\,f \equiv_\beta f(Y\,f) \equiv_\beta f(f(Y\,f)) \equiv_\beta \cdots$처럼
함수 $f$를 몇 번이고 반복해서 적용하는 계산을 표현할 수 있다.

Y조합자를 드브루인 인덱스를 활용하는 람다식을 나타내는 하스켈 데이터 타입인 `DExpr`로
표현하여 다음과 같이 `yC`로 선언하자.

In [13]:
yC = DAbs (DApp e211 e211) -- \f.(\x.f(x x))(\x.f(x x))
   where e211 = DAbs (DApp (DI 2) (DI 1 `DApp` DI 1))

In [14]:
yC -- 드브루인 인덱스로 나타낸 Y조합자

\(\2 (1 1)) (\2 (1 1))

Y조합자를 항등함수($\lambda z.z$)에 적용하면 세 작은걸음만에 앞서 살펴본 계산이 끝나지 않는 람다식을 유도할 수 있다. 
\vspace*{-2ex}
\begin{align*}
(\lambda f.(\lambda x.f\,(x\;x))\,(\lambda x.f\,(x\;x)))~(\lambda z.z)
\longmapsto\;& (\lambda x.(\lambda z.z)(x\;x))\,(\lambda x.(\lambda z.z)(x\;x)) \\
\longmapsto\;& (\lambda x.x\;x)\,(\lambda x.(\lambda z.z)(x\;x)) \\
\longmapsto\;& (\lambda x.x\;x)\,(\lambda x.x\;x)
\end{align*}
산술식에서처럼 `step`을 확장한 `step'`를 고차함수 `hat`을 통해 선언하자.

In [15]:
import Data.List (nub)
hat f es = nub (concat [f e| e <- es])

step' :: [DExpr] -> [DExpr]
step' = hat step

In [16]:
DApp yC (DAbs (DI 1)) -- yC (\x.x)

(\(\2 (1 1)) (\2 (1 1))) (\1)

`step'` 함수를 세 번 연달아 적용한 결과로 나오는 리스트의 모든 원소를
다음과 같이 한줄에 하나씩 출력해 보자. 앞서 `step` 함수를 제대로 완성했다면,
아래의 출력 결과 중에 계산이 끝나지 않는 람다식
$(\lambda x.x~x)~(\lambda x.x~x)$에 해당하는 `(\1 1) (\1 1)`도
출력됨을 확인할 수 있을 것이다. 참고로 리스트의 원소들이 출력되는
순서는 `step`을 작성한 방식에 달라질 수도 있다.

In [17]:
-- yC (\x.x)로부터 세 작은걸음 진행한 모든 람다식을 한줄에 하나씩 출력
mapM_ print $ (step' . step' . step') [ DApp yC (DAbs (DI 1)) ]

(\(\1) (1 1)) (\(\1) (1 1))
(\1) ((\1) ((\(\1) (1 1)) (\(\1) (1 1))))
(\1) ((\1 1) (\(\1) (1 1)))
(\1) ((\(\1) (1 1)) (\1 1))
(\1 1) (\1 1)
(\1) ((\1 1) (\1 1))
(\1 (1 (1 ((\2 (1 1)) (\2 (1 1)))))) (\1)

## 하스켈에서 고정점 조합자 선언하기
타입없는 람다계산법에서 자기 자신에 대한 적용(self-application)을
이용해 끝나지 않는 계산을 표현하거나 Y조합자와 같은 고정점 조합자를
람다식으로 정의할 수 있음을 알아보았다. 그렇지만 하스켈에서는 순수한
람다식만으로 끝나지 않는 계산이나 고정점 조합자를 표현하기는 어렵다.
왜냐하면 하스켈은 타입있는 람다계산법을 기반으로 하는데,
타입있는 람다계산법에서는 자기 자신에 대한 적용 $x~x$에
적절한 타입을 부여하기 어렵기 때문이다. 인자 부분인 오른쪽의
$x:\tau$라면 함수 부분인 왼쪽의 $x:\tau\to\tau$여야 한다.
단순타입 람다계산법(STLC)에서는 같은 식의 타입이
$\tau$이면서 $\tau\to\tau$이기도 하기란 불가능하다. 앞으로 소개할
다형 람다계산법에서는 자기 자신에 대한 적용에 타입을 부여할 수는 있지만
무한한 계산을 표현할 수는 없다. 애초에 타입있는 람다계산법을 고안하게 된 동기가
바로 끝없는 계산처럼 논리적 모순에 대응되는 람다식을 허용하지 않기 위해서다.
그래서 튜링완전한 범용 함수형 언어에서는 람다식과 별도로
재귀적 표현을 위한 문법요소를 제공한다.

In [18]:
:type \x -> x x  -- 하스켈에서 self-application 람다식은 타입 오류

: 

그런데 만일 하스켈에서 딱 한 번만 재귀함수를 선언할 수 있고 다른 곳에서는
재귀적인 표현을 허용하지 않도록 제약을 가한다면 우리는 어떤 재귀함수를 선언해야 할까?
다음과 같이 고정점 조합자를 딱 한번만 선언하면 다른 곳에서 재귀적 표현이 금지되더라도
일반적인 재귀함수를 얼마든지 만들어낼 수 있다.

In [19]:
fix f = f (fix f) -- 고정점 조합자의 성질을 재귀함수로 선언

In [20]:
:type fix

fix :: forall t. (t -> t) -> t

정수 $n$을 받아 팩토리얼 $n!$를 계산하는 함수를 두 가지 방법으로 선언하여 비교해 보자.
처음에는 직접 재귀함수로 선언하 보고, 그 다음에는
고정점 조합자 `fix`와 다른 재귀적이지 않은 함수를 활용해 선언해 보겠다.
프로그램의 단순화를 위해 팩토리얼 함수를 음수에 적용하는 것은 고려하지 않고
0이상의 자연수에만 적용한다고 가정하자.

In [21]:
fact :: Integer -> Integer -- 직접 재귀함수로 팩토리얼 선언
fact 0 = 1                 -- 0! = 1
fact n = n * fact (n-1)    -- n! = n * (n-1)!

In [22]:
fact 0
fact 5

1

120

In [23]:
factF :: (Integer -> Integer) -> (Integer -> Integer)
factF f 0 = 1           -- f를 원래 파라메터 앞에 추가
factF f n = n * f (n-1) -- 재귀함수 호출 위치에 f 사용

fact' = fix factF -- 고정점 조합자를 활용한 팩토리얼 선언

In [24]:
:type fact'

fact' :: Integer -> Integer

In [25]:
fact' 0
fact' 5

1

120

# 다형 람다계산법
\label{sec:SystemF}

문법구조\vspace{-4.5ex}
\begin{align*}
\qquad
t \in B & ~\text{\small(기본타입)} &
\tau\in T ~::= ~& t ~\mid~ \tau \to \tau
~\mid~ \alpha ~\mid~ \forall \alpha.\tau
\\
c \in C & ~\text{\small(기본상수)} &
e\in E ~::=~ & c \mid x \mid \lambda x{\,:\,}\tau.\,e \mid e~e
\mid \Lambda \alpha.\,e \mid e\,@\tau
\end{align*} 타입규칙
$\qquad
\Delta ~::=~ c{\,:\,}\tau,\Delta \mid \bm{\cdot} \qquad\qquad\qquad
\Gamma ~::=~ \alpha{\,:\,}\star,\Gamma \mid x{\,:\,}\tau,\Gamma \mid \bm{\cdot}$
$$
{\scriptstyle(\textsc{Con})}
\frac{~ ~}{~ \Gamma \vdash c:\tau ~}
({\scriptstyle c{\,:\,}\tau\;\in\,\Delta})
\qquad\qquad
{\scriptstyle(\textsc{Var})}
\frac{~ ~}{~ \Gamma \vdash x:\tau ~}
({\scriptstyle x{\,:\,}\tau\;\in\,\Gamma})
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{Abs})}
\frac{~ x:\tau_2,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \lambda x{\,:\,}\tau_2.\,e : \tau_2\to\tau ~}
\qquad
{\scriptstyle(\textsc{App})}
\frac{~ \Gamma \vdash e_1:\tau_2\to\tau ~\quad~ \Gamma \vdash e_2:\tau_2 ~}{~ \Gamma \vdash e_1\;e_2 : \tau ~}
$$ \vspace*{-.5ex}
$$
{\scriptstyle(\textsc{TAbs})}
\frac{~ \alpha{\,:\,}\star,\Gamma \vdash e:\tau ~}{~ \Gamma \vdash \Lambda \alpha.\,e : \forall \alpha.\tau ~}
\qquad
{\scriptstyle(\textsc{TApp})}
\frac{~ \Gamma \vdash e:\forall\alpha.\tau ~}{~ \Gamma \vdash e\,@\tau_1 : \{\alpha{\mapsto}\tau_1\}\tau ~}
$$
$~$\newline
의미구조
$\displaystyle
 \quad {\scriptstyle(\beta_\to)}\frac{~~}{~(\lambda x{:}\tau.\,e)\,e_2 \longmapsto \{x{\mapsto}e_2\}e~}
\qquad {\scriptstyle(\beta_\forall)}\frac{~~}{~(\Lambda \alpha.\,e)\,@\tau \longmapsto \{\alpha{\mapsto}\tau\}e~}$
$$ \mathcal{E} ~::=~ \bullet
\mid \lambda x{\,:\,}\tau.\,\mathcal{E} 
\mid \mathcal{E}~e \mid e~\mathcal{E}
\mid \Lambda \alpha.\,\mathcal{E} \mid \mathcal{E}\,@\tau
\qquad
{\scriptstyle(\mathcal{E})}\frac{~e\longmapsto e'~}{~\mathcal{E}[e]\xmapsto{~_C~}\mathcal{E}[e']~}
$$

프랑스의 논리학자인 장-이브 지라르\cite{Girard1972SystemF}는 System F라는 이름의,
미국의 컴퓨터 과학자 존 레이놀즈\cite{Reynolds1974PolyLambda}는
이후 다형 람다계산법(polymorphic $\lambda$-calculus) 혹은
다형타입 람다계산법(polymoprphically typed $\lambda$-calculus)으로 불리게 된,
알고보니 똑같은 타입있는 람다계산법(그림 \ref{fig:SystemF})을 1970년대 초반에 각자 독립적으로 고안했다.
\`다형 람다계산법'보다는 \`System F'가 더 짧으므로 주로 System F로 부르겠다.
System F의 타입은 STLC의 타입을 구성하는 요소 외에도 타입변수($\alpha$)와 다형타입($\forall\alpha.\tau$)을 포함한다.
System F의 식은 STLC의 식을 구성하는 요소 외에도 타입요약식(type abstraction)과 타입적용식(type application)을 포함한다. 함수요약식($\lambda x{:}\tau.e$)이 식을 식에 대응시키는 함수를 표현한다면
타입요약식($\Lambda\alpha.e$)은 타입을 식에 대응시킨키는 함수를 표현한다.
식을 식에 적용하는 함수적용식($e_1~e_2$)과의 차이를 뚜렷이 드러내기 위해
식을 타입 적용하는 타입적용식($e\;@\tau$)에서는 인자로 넘기는 타입 앞에 $@$기호를 표기한다.
참고로, 표준적인 타입적용식의 표기는 레이놀즈의 논문\cite{Reynolds1974PolyLambda}에서처럼
$e[\tau]$이다. 하지만 실행맥락의 구멍을 채워넣는 표현($\mathcal{E}[e]$)에도 같은 괄호를 사용하므로,
여기서는 혼동의 여지를 줄이고자, 하스켈에서 명시적인 타입 적용을 표현하도록 지원하는
GHC확장 기능의 문법을 따라 $e\;@\tau$와 같이 표기하였다.

타입요약식에 대한 타입규칙($\scriptstyle\textsc{TAbs}$)은 타입요약식이 다형타입이 됨을 규정한다.
타입적용식에 대한 타입규칙($\scriptstyle\textsc{TApp}$)은
왼쪽의 적용하는 식($e$)이 다형타입($\Lambda\alpha.\tau$)이어야 하며
전체 타입적용식($e\,@\tau_1$)의 타입은 $\tau$에 자유로이 나타나는
$\alpha$를 오른쪽의 타입 인자 $\tau_1$로 구체화한 타입이 됨을 규정한다.
그 외의 다른 타입규칙은 STLC에서와 같다.

그림 \ref{fig:SystemF}의 의미구조는 $\beta_\to$규칙과 $\beta_\forall$규칙을
적용하는 순서를 제약하지 않는 단순한 형태의 의미구조이다. System F를 바탕으로 하는
정적 타입 함수형 언어의 실제 구현은 실행 전의 타입 검사 단계에서
$\beta_\forall$규칙만을 미리 적용해 타입을 가능한 한 구체화해 놓고,
실행 시간에 $\beta_\to$규칙을 적용하며 계산을 진행하는 것으로 이해할 수 있다. 

## 다형 람다계산법과 하스켈

In [26]:
:type id  -- 표준라이브러리 항등함수 id는 다형타입

id :: forall a. a -> a

In [27]:
:type id @Int     -- Int에 명시적 타입 적용
:type id @String  -- String에 명시적 타입 적용

id @Int :: Int -> Int

id @String :: String -> String

In [28]:
id      3 -- 타입 인자가 생략되어 있어도 타입유추로 알아서 척척
id @Int 3

3

3

In [29]:
id         "hello" -- 타입 인자가 생략되어 있어도 타입유추로 알아서 척척
id @String "hello"

"hello"

"hello"

In [30]:
id @Int "hello" -- 명시적으로 구체화한 타입과 다르게 사용하면 타입 오류

: 

위에서 살펴본 바와 같이 하스켈에서 다형 함수 적용은 System F와 사실상 같은 문법이다.
하스켈은 타입유추를 잘 지원하므로 대부분의 경우 명시적 타입적용을 생략하더라도
컴파일러가 이후에 연이어 적용하는 인자로부터 필요한 타입 정보를 얻어 드러나 보이지 않게 타입적용이 수행된다.

한편, 하스켈에는 타입요약식($\Lambda \alpha.e$)에 직접적으로 대응되는 문법요소는 없다.
다만 아래와 같이 함수 선언에 앞서 `myid :: forall a. a -> a` 또는 줄여서 `myid :: a -> a`로
다형타입을 선언함으로써
`myid`를 System F의 타입요약식으로 선언한 것과 같은 효과를 볼 수 있다.
참고로, 타입 선언 없이도 마찬가지 다형타입이 유추되지만
`@`기호를 사용하는 명시적 타입적용은 사용할 수 없다.

In [31]:
myid :: a -> a
myid = \x -> x

In [32]:
:type myid
:type myid @Int
:type myid @String

myid :: forall a. a -> a

myid @Int :: Int -> Int

myid @String :: String -> String

## 람다계산법과 데이터 타입

$\begin{array}{rl}
\texttt{true}  ~=& \lambda x.\lambda y.x \\
\texttt{false} ~=& \lambda x.\lambda y.y \\
\texttt{ifte}  ~=& \lambda b.\lambda x.\lambda y.b~x~y ~\equiv_\eta~ \lambda b.b
\end{array}$

타입없는 람다계산법은 튜링기계와 마찬가지로 아무 계산이나 다 표현할 수 있다는 처치--튜링 명제에 대해 언급한 바 있다.
우리가 흔히 접하는 프로그래밍언어에서는 진리값이나 정수와 같은 기본 타입을 비롯해
순서쌍이나 리스트 등의 복합적 데이터 타입을 다루는 계산을 한다.
그런데 함수만 있는 람다계산법이 어떻게 이런 모든 계산을 다룰 수 있다는 것일까?
람다계산법의 창시자인 알론조 처치는 데이터를 함수로 표현하는 방법을 발견하였는데
이를 처치 인코딩(Church encoding)이라 한다. 이해하기 좋은 기초적인 사례로
진리값 상수에 대한 처치 인코딩(그림 \ref{fig:ChurchBool})을 살펴보자.
프로그래밍언어에서 진리값을 활용하는 대표적인 요소는 `if` $b$ `then` $e_1$ `else` $e_0$와
같은 형태의 바로 조건식(conditional expression)인데, 이를 람다식의 함수 `ifte`로 생각하면
`ifte` $b$ $e_1$ $e_0$에서 $b$가 `true`로 계산될 경우는 $e_1$과 같도록 $b$가 `false`로
계산될 경우는 $e_0$과 같이 동작해야 한다. 처치 인코딩의 핵심 아이디어는 데이터를 사용하는
대표적인 연산의 실제 의미를 데이터 상수 그 자체에 담아놓자는 것이다. 진리값의 경우
조건식의 유의미한 계산은 진리값 상수를 인코딩한 `true`나 `false`에 담겨있다는 말이다.
그래서 조건식을 표현하는 `ifte`는 항등함수의 형태로 사실상 유의미한 계산 없이
진리값 상수를 그대로 내어주도록 (즉, `ifte` $b$ $e_1$ $e_0$ $\equiv_\beta$ $b~e_1~e_0$)
인코딩되어 잇다. 이를 우리가 잘 아는 조건식의 동작에 맞추려면,
`true` $e_1$ $e_0$ $\equiv_\beta$ $e_1$이고
`false` $e_1$ $e_0$ $\equiv_\beta$ $e_0$가 되도록,
진리값 상수 `true`를 $\lambda x.\lambda y.x$로
`false`를 $\lambda x.\lambda y.y$로 인코딩하면 된다.

그런데 단순타입 람다계산법(STLC)에서는 처치--인코딩과 같이 데이터 상수를 별도로 설정하지 않고
순수한 람다식만으로 인코딩해 활용하기는 곤란하다.
조건식을 나타내는 `ifte`에 $\texttt{BOOL}\to T\to T$와 같은 타입을 부여하고
그에 맞춰 `true`와 `false`도 $\lambda x{:}T.\lambda y{:}T.x$와 $\lambda x{:}T.\lambda y{:}T.y$로 인코딩해야 하는데,
문제는 `T`를 어떤 하나로 정하면 다른 타입에 대한 조건식을 작성할 수 없게 된다.
예를 들어 $\texttt{ifte} : \texttt{BOOL}\to \texttt{Int}\to \texttt{Int}$로 한다면
정수를 계산하는 조건식은 작성할 수 있지만 문자열을 계산하는 조건식은 작성할 수 없다.
굳이 억지로 이렇게 해보겠다면\newline\indent
$\texttt{ifte}_\texttt{Int} : \texttt{BOOL}_\texttt{Int}\to \texttt{Int}\to \texttt{Int}$ 및 $\texttt{true}_\texttt{Int}$와 $\texttt{false}_\texttt{Int}$,\newline\indent
$\texttt{ifte}_\texttt{String} : \texttt{BOOL}_\texttt{String}\to \texttt{String}\to \texttt{String}$ 및 $\texttt{true}_\texttt{String}$와 $\texttt{false}_\texttt{String}$,\newline\indent
$~\vdots~$\newline
이렇게 계산하려는 각각의 타입 $T$마다
$\texttt{ifte}_T$ 및 $\texttt{true}_T$와 $\texttt{false}_T$로
$\texttt{BOOL}_T$ 타입을 한벌씩 따로 인코딩해야 하므로
불편하고 혼란스러울 따름이다.

다형 람다계산법에서는 모든 타입에 대해 일관되게 동작을 하는 식에 하나의 다형타입($\forall \alpha.\tau$)을
부여하는 것이 가능하며, 다형타입이 부여된 하나의 식을 필요에 따라 타입적용을 통해 각각의 타입으로 구체화하여 활용할 수 있다.
다형 람다계산법에서와 같은 다형타입로 표현할 수 있는 이러한 특성을 \`매개변수 다형성'(parametric polymorphism)이라 하며,
이를 이용하면 다형 람다계산법에서는 처치--인코딩과 같이 순수한 람다식만으로 데이터를 표현하는 것이 가능하다.
TODO cite
Corrado Boehm and Alessandro Berarducci (1985) Automatic Synthesis of Typed Lambda-Programs on Term Algebras. Theoretical Computer Science, v39:135–154


In [33]:
type BOOL = forall a. a -> a -> a 
false, true :: BOOL
false = \x -> \y -> y
true  = \x -> \y -> x

In [34]:
ifte :: BOOL -> a -> a -> a
ifte = \b -> b

In [35]:
andB, orB :: BOOL -> BOOL -> BOOL
andB = \b1 -> \b2 -> b1 b2 false
orB =  \b1 -> \b2 -> b1 true b2

notB :: BOOL -> BOOL
notB = \b -> \x -> \y -> b y x

\section*{요점정리}
* TODO
* TODO

\section*{연습문제}
1. TODO
1. TODO

\section*{탐구과제}
1. TODO
1. TODO